# 06: Graph RAG

Implement knowledge graph-based retrieval.

**Learning Objectives:**
- Build knowledge graphs
- Graph traversal
- Multi-hop reasoning


In [ ]:
import sys
sys.path.insert(0, '..')

from src.graph_rag import GraphRAG

print('✓ Imports successful')

## Knowledge Graph

TODO: Implement entity extraction

In [ ]:
graph = GraphRAG()

# Query using graph
response = graph.query('What rights does a data subject have?')
print(f'Response: {response}')

# Multi-hop reasoning
response2 = graph.query('How is GDPR related to data subjects?', use_multi_hop=True)
print(f'\nMulti-hop: {response2}')